In [11]:
import os
from dotenv import load_dotenv
from custom_prompts import CustomPrompts
import phoenix as px
import json
from pprint import pprint
import nest_asyncio
import pandas as pd
from llama_index.core import set_global_handler, Settings, PromptTemplate, ChatPromptTemplate
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.embeddings.gemini import GeminiEmbedding
from llama_index.core.llms import ChatMessage, MessageRole
from llama_index.llms.openai import OpenAI
from llama_index.llms.gemini import Gemini
from llama_index.core.readers import SimpleDirectoryReader
from llama_index.core.indices import VectorStoreIndex
from llama_index.core.node_parser import SentenceSplitter, SemanticSplitterNodeParser
from llama_index.core.tools import QueryEngineTool, ToolMetadata, FunctionTool
from llama_index.core.query_engine import SubQuestionQueryEngine, PandasQueryEngine
from llama_index.core.indices.struct_store import JSONQueryEngine
from llama_index.core.agent import ReActAgent, AgentRunner
from llm_compiler_agent_pack.llama_index.packs.agents_llm_compiler.step import (
    LLMCompilerAgentWorker,
)

nest_asyncio.apply()
load_dotenv()
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [67]:
session = px.launch_app()
set_global_handler("arize_phoenix")

Existing running Phoenix instance detected! Shutting it down and starting a new instance...


🌍 To view the Phoenix app in your browser, visit http://localhost:6006/
📺 To view the Phoenix app in a notebook, run `px.active_session().view()`
📖 For more information on how to use Phoenix, check out https://docs.arize.com/phoenix


In [73]:
Settings.llm = Gemini(max_tokens=256, temperature=0.2)
# Settings.llm = OpenAI(model="gpt-3.5-turbo", max_tokens=256, temperature=0.1)

Settings.embed_model = (
    OpenAIEmbedding()
    if isinstance(Settings.llm, OpenAI)
    else GeminiEmbedding(model_name="models/text-embedding-004")
)

In [4]:
papers = SimpleDirectoryReader(input_dir="input/papers/").load_data()
syllabus = SimpleDirectoryReader(input_dir="input/syllabus/").load_data()
markschemes = SimpleDirectoryReader(input_dir="input/markschemes/extracted").load_data()

In [ ]:
# Experimental
splitter = SemanticSplitterNodeParser(
    buffer_size=2, breakpoint_percentile_threshold=99, embed_model=Settings.embed_model
)
paper_nodes = splitter.get_nodes_from_documents(papers)
for node in paper_nodes:
    print(node)
    print("\n")

In [5]:
papers_index = VectorStoreIndex.from_documents(
    papers,
    # transformations=[
    #     SentenceSplitter(
    #         chunk_size=200,
    #         chunk_overlap=20,
    #     )
    # ],
    show_progress=True,
)

# syllabus_index = VectorStoreIndex.from_documents(
#     syllabus,
#     transformations=[
#         SentenceSplitter(
#             chunk_size=200,
#             chunk_overlap=20,
#         )
#     ],
#     show_progress=True,
# )

Parsing nodes:   0%|          | 0/5 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/5 [00:00<?, ?it/s]

# With JSON Query Engine

In [47]:
papers_json_engine = JSONQueryEngine(
    json_value=json.loads(open("input/markschemes/extracted/may2017.json").read()),
    json_schema=json.loads(open("input/markschemes/extracted/exam_schema.json").read()),
    llm=Settings.llm,
    json_path_prompt=CustomPrompts.IB_PAPERS_JSON_PROMPT,
    show_progress=True,
    synthesize_response=False,
)
papers_json_engine.get_prompts()

{'json_path_prompt': PromptTemplate(metadata={'prompt_type': <PromptType.CUSTOM: 'custom'>}, template_vars=['schema', 'query_str'], kwargs={}, output_parser=None, template_var_mappings=None, function_mappings=None, template='We have provided a JSON schema below:\n{schema}\nGiven a task, respond with a JSON Path query that can retrieve data from a JSON value that matches the schema.\nProvide the JSON Path query in the following format: \'JSONPath: <JSONPath>\'\nYou must include the value \'JSONPath:\' before the provided JSON Path query. Example Format:\nTask: What is the first question in May 2020 paper?\nResponse: JSONPath: $.["May 2020"]["questions"]["1"]["question_text"]\n\nTask: From November 2019 paper provide the marscheme for question 2.\nResponse: JSONPath: $.["November 2019"]["questions"]["2"]["markscheme"]\n\nLet\'s try this now: \n\nTask: {query_str}\nResponse: '),
 'response_synthesis_prompt': PromptTemplate(metadata={'prompt_type': <PromptType.SQL_RESPONSE_SYNTHESIS: 'sql_

In [48]:
response = papers_json_engine.query("How many marks are awarded to question 7 in May 2017 exam?")
print(response.metadata["json_path_response_str"])
print(response)

JSONPath: $.["May 2017"]["questions"]["7"]["maximum_points"]
{"[\"May 2017\"][\"questions\"][\"7\"][\"maximum_points\"]": "3"}


In [49]:
response = papers_json_engine.query("question 3 markscheme in May 2017 exam")
print(response.metadata["json_path_response_str"])
print(response.response)

JSONPath: $.["May 2017"]["questions"]["3"]["markscheme"]
{"[\"May 2017\"][\"questions\"][\"3\"][\"markscheme\"]": "Award up to [2 max].\nTo patch any vulnerabilities/bugs/cyberspace threats;\nTo provide improved functionality/new functions/usability/maximize efficiencies;*\n\nTo generate income for the software company/to innovate and stay ahead of other software companies;\n\nTo ensure compatibility with other (updated) technologies; [2]"}


# Single Query Engine on Papers

In [90]:
papers_vector_engine = papers_index.as_query_engine(
    streaming=True,
    similarity_top_k=3,
)
papers_vector_engine.update_prompts(
    {"response_synthesizer:text_qa_template": CustomPrompts.IB_PAPERS_RAG_PROMPT}
)

In [126]:
response = papers_vector_engine.query(
    "Which question is about MAC addresses and how many marks does it carry?"
)
response.print_response_stream()

Question 8 is about MAC addresses and carries 3 marks.

In [197]:
response = papers_vector_engine.query(
    "Retrieve question 3 from the paper."
)
response.print_response_stream()

The provided context does not contain question 3.

# Single Query Engine on Syllabus

In [12]:
syllabus_pd_engine = syllabus_index.as_query_engine(streaming=True)

In [13]:
response = syllabus_pd_engine.query("What is the topic with number for Describe how communication over networks is broken down into different layers.")
response.print_response_stream()

3.1.3

In [14]:
response = syllabus_pd_engine.query("Topic with number 3.1.3")
response.print_response_stream()

The provided context does not contain any information about topic 3.1.3.

In [170]:
nodes = syllabus_index.as_retriever().retrieve(
    "Topic with number for 'Evaluate the use of a VPN.'"
)
for node in nodes:
    print(node.get_text())

3.1.3Describe how communication over networks is broken down into different layers.2Awareness of the OSI seven layer model is required, but an understanding of the functioning of each layer is not.3.1.4Identify the technologies required to provide a VPN.23.1.5Evaluate the use of a VPN.3S/E, AIM 9 The use of a VPN has led to changes in working patterns.
11Explain how data is transmitted by packet switching.3Wireless networking3.1.12Outline the advantages and disadvantages of wireless networks.2S/E wireless networks have led to changes in working patterns, social activities and raised health issues.3.1.13Describe the hardware and software components of a wireless network.23.1.14Describe the characteristics of wireless networks. 2Include: WiFi; Worldwide Interoperability for Microwave Access (WiMAX); 3G mobile; future networks.S/E, INT Connectivity between different locations.3.1.15Describe the different methods of network security.2Include encryption types, userID, trusted media access c

The retriever fails to locate assessment statements from topic numbers like `3.1.3`. We need to provide syllabus in a more structured format. Details can be found in [Syllabus Preprocessing Notebook](syllabus_preprocessing.ipynb).

# PandasQueryEngine for Syllabus

In [34]:
syllabus_pd = pd.read_excel("input/syllabus/extracted/syllabus.xlsx", index_col=0)
syllabus_pd = syllabus_pd[["Assessment Statement"]]
syllabus_pd.head()

,Assessment Statement
Topic Number,
1.1.1,Identify the context for which a new system is...
1.1.2,Describe the need for change management.
1.1.3,Outline compatibility issues resulting from si...
1.1.4,Compare the implementation of systems using a ...
1.1.5,Evaluate alternative installation processes.


In [48]:
syllabus_pd_engine = PandasQueryEngine(
    df=syllabus_pd,
    head=3,
    response_synthesis_prompt=CustomPrompts.IB_SYLLABUS_PANDAS_RESPONSE_PROMPT,
    output_kwargs={
        "max_colwidth": None
    },  # from reading: https://github.com/run-llama/llama_index/blob/main/llama-index-experimental/llama_index/experimental/query_engine/pandas/output_parser.py
    synthesize_response=True,
    verbose=False,
)

In [169]:
response = syllabus_pd_engine.query(
    # "What is topic 3.1.3 about and what subcategory does it fall in?",
    # "How many topics are in User focus subcatergory?",
    "Compare topic 3.1.3 and 3.1.4.",
)
print(response.response)

Topic 3.1.3 focuses on the layered structure of network communication, while topic 3.1.4 specifically addresses the technologies used to establish a VPN.


In [94]:
response = syllabus_pd_engine.query(
    "3.1.3"
)
print(response.response)

From syllabus: Describe how communication over networks is broken down into different layers.


In [161]:
print(response.metadata)

{'pandas_instruction_str': 'df.query("`Topic Number` == \'3.1.3\'")', 'raw_pandas_output': '                       Subcategory                                                            Assessment Statement\nTopic Number                                                                                                      \n3.1.3         Network fundamentals  Describe how communication over networks is broken down into different layers.'}


# Agents

In [51]:
questions_json = json.loads(open(f"input/markschemes/extracted/may2017.json").read())


def retrieve_question(exam: str, question_number: int, with_maximum_points: bool = False, with_markscheme: bool = False):
    """
    Retrieve a specific question from an exam. with_ arguments determine whether those fields are included in the response.
    """
    question = (
        questions_json.get(exam).get("questions").get(str(question_number)).copy()
    )
    if not with_markscheme:
        question.pop("markscheme")
    if not with_maximum_points:
        question.pop("maximum_points")
    return question

def topic_number_to_assessment_statement(topic_number: str):
    """ Retrieve the assessment statement for a topic number."""
    return syllabus_pd.loc[topic_number, "Assessment Statement"]

In [123]:
query_engine_tools = [
    QueryEngineTool(
        query_engine=papers_vector_engine,
        metadata=ToolMetadata(
            name="papers",
            description=(
                "Searches in the raw/unstructured text of IB CS papers.\n"
                "Only used for semantic search on question content, and not for lookups like 'topic 2.1.1'.\n"
                "Examples: 'Which question is about MAC addresses?', "
                "'Question that asks to trace an algorithm from May 2017'.\n"
            ),
        ),
    ),
    QueryEngineTool(
        query_engine=syllabus_pd_engine,
        metadata=ToolMetadata(
            name="syllabus_pd_engine",
            description=(
                "Searches for assessment statements given the syllabus topic number.\n"
                "Examples: 'What topic is 3.1.3 about?', 'Subcategory of topic 2.1.1.', "
                "'What topics are in User focus subcategory?'."
            ),
        ),
    ),
    FunctionTool.from_defaults(
        fn=retrieve_question,
    ),
    # FunctionTool.from_defaults(
    #     fn=topic_number_to_assessment_statement,
    # ),
    # QueryEngineTool(
    #     papers_json_engine,
    #     metadata=ToolMetadata(
    #         name="papers_json_engine",
    #         description=(
    #             "Searches for specific exam paper questions and markschemes using plain text.\n"
    #             "Examples: 'What is the marksheme for question 3 from May 2020?', 'How many points is November 2015 paper question three?', "
    #             "'Retrieve the first question in May 2017 exam.'"
    #         ),
    #     ),
    # ),
]

In [ ]:
react_agent = ReActAgent.from_tools(
    tools=query_engine_tools,
    verbose=True,
    # chat_history=[
    #     ChatMessage(
    #         role=MessageRole.SYSTEM,
    #         content="Beware, observations are not users questions or queries, you shall not respond to them.",
    #     )
    # ],
    context=CustomPrompts.REACT_CONTEXT,
)

# Optional: Update the system prompts
# react_agent.update_prompts(
#     {"agent_worker:system_prompt": CustomPrompts.REACT_SYSTEM_PROMPT}
# )

In [37]:
# Experimental agents
s_engine = SubQuestionQueryEngine.from_defaults(
    query_engine_tools=query_engine_tools,
    use_async=False,
    verbose=True,
)

llm_compiler = AgentRunner(
    LLMCompilerAgentWorker.from_tools(
        query_engine_tools,
        verbose=True,
        llm=Settings.llm,
    ),
)

# Querying ReAct Agent

In [128]:
response = react_agent.query(
    "Retrieve question 9 from the May 2017 paper."
)
print(response)

Thought: The current language of the user is: english. I need to use a tool to help me answer the question.
Action: retrieve_question
Action Input: {'exam': 'May 2017', 'question_number': 9}
Observation: {'question_text': 'Outline the main steps involved in a selection sort. [3]'}
Thought: I can answer without using any more tools. I'll use the user's language to answer
Answer: The main steps involved in a selection sort are:
1. Find the minimum element in the unsorted portion of the list.
2. Swap the minimum element with the leftmost unsorted element.
3. Repeat steps 1 and 2 until the entire list is sorted.
The main steps involved in a selection sort are:
1. Find the minimum element in the unsorted portion of the list.
2. Swap the minimum element with the leftmost unsorted element.
3. Repeat steps 1 and 2 until the entire list is sorted.


In [217]:
response = react_agent.query(
    "Mark my answer to question 5 from May 2017 paper using the official markscheme, show why you awarded each mark: The operating system prevents the system from crashing when more than one program is run at the same time by managing the allocation and deallocation of memory to each program. This ensures that each program has its own dedicated memory space and cannot access the memory of other programs. Additionally, the operating system uses techniques such as virtual memory and memory protection to prevent programs from accessing memory that they are not authorized to access."
)
print(response)

Thought: The current language of the user is: english. I need to use a tool to help me answer the question.
Action: retrieve_question
Action Input: {'exam': 'May 2017', 'question_number': 5, 'with_markscheme': True}
Observation: {'question_text': 'One of the functions of an operating system is memory management.\n\nDescribe how this function prevents the system from crashing when more than one program is run at the same time. [2]', 'markscheme': 'The OS allocates (and deallocates) specific sections of memory to each\nprogram/process/module;\nThis ensures that the memory assigned to one program is not overwritten;\nUses secondary/virtual memory to allow more processes to run simultaneously;\n\n*(Note:  Do not accept vague reasons). [2]'}
Thought: I can answer without using any more tools. I'll use the user's language to answer
Answer: The operating system prevents the system from crashing when more than one program is run at the same time by managing the allocation and deallocation of m

In [101]:
response = react_agent.query(
    "Retrieve questions from the paper are about logic gates and truth tables."
)
print(response)

Thought: The current language of the user is: english. I need to use a tool to help me answer the question.
Action: papers
Action Input: {'input': 'logic gates and truth tables'}
Observation: (c) (i)  Construct the truth table corresponding to this diagram. [3]
  (ii)  Identify the  single logic gate that is equivalent to this diagram . [1]
Thought: I can answer without using any more tools. I'll use the user's language to answer
Answer: (c) (i) Construct the truth table corresponding to this diagram. [3]
(ii) Identify the single logic gate that is equivalent to this diagram . [1]
(c) (i) Construct the truth table corresponding to this diagram. [3]
(ii) Identify the single logic gate that is equivalent to this diagram . [1]


In [71]:
response = react_agent.query(
    "In May 2017 paper what is the total marks awarded to questions about logic gates and truth tables?"
)
print(response)

Thought: The current language of the user is: english. I need to use a tool to help me answer the question.
Action: papers
Action Input: {'input': 'May 2017 paper logic gates and truth tables'}
Observation: (c) (i) Construct the truth table corresponding to this diagram. [3]
(ii) Identify the single logic gate that is equivalent to this diagram. [1]
(d) Outline how truth tables can be used to test that any two logic diagrams are equivalent. [2]
Thought: I can answer without using any more tools. I'll use the user's language to answer
Answer: The total marks awarded to questions about logic gates and truth tables in the May 2017 paper is 6 marks.
The total marks awarded to questions about logic gates and truth tables in the May 2017 paper is 6 marks.


In [122]:
response = react_agent.query(
    "What does the pseudocode do in the coding question from May 2017 paper?"
)
print(response)

Thought: The current language of the user is: english. I need to use a tool to help me answer the question.
Action: papers
Action Input: {'input': 'May 2017 paper pseudocode'}
Observation: yyinput the number, S, to be searched for
yyread in the values from the NUMBERS  collection into the array D.  Note : you can 
assume that the array is large enough and that the collection is not empty
yyperform a linear search for S on the array D  
yyoutput the message “ found ” or “not found ” as appropriate.
Thought: I can answer without using any more tools. I'll use the user's language to answer
Answer: The pseudocode performs a linear search for a number, S, in an array D. It reads the values from the NUMBERS collection into the array D, and then performs a linear search for S on the array D. If S is found, it outputs the message "found", otherwise it outputs the message "not found".
The pseudocode performs a linear search for a number, S, in an array D. It reads the values from the NUMBERS co

In [126]:
response = react_agent.query(
    "What is syllabus topic 2.1.2 about? Give an example of a question that can be asked from this topic."
)
print(response)

Thought: The current language of the user is: english. I need to use a tool to help me answer the question.
Action: syllabus_pd_engine
Action Input: {'input': '2.1.2'}
Observation: Primary memory, also known as main memory or RAM (Random Access Memory), is the computer's short-term memory that stores data and instructions that are currently being processed by the CPU. It is volatile, meaning that data is lost when the computer is turned off. Primary memory is much faster than secondary memory (such as hard drives), but it is also more expensive.
Thought: I can answer without using any more tools. I'll use the user's language to answer
Answer: Syllabus topic 2.1.2 is about primary memory. An example of a question that can be asked from this topic is: "Explain the difference between primary memory and secondary memory."
Syllabus topic 2.1.2 is about primary memory. An example of a question that can be asked from this topic is: "Explain the difference between primary memory and secondary 

# Chat with ReAct Agent

In [119]:
react_agent.chat_repl()

===== Entering Chat REPL =====
Type "exit" to exit.

Thought: The current language of the user is: english. I need to use a tool to help me answer the question.
Action: retrieve_question
Action Input: {'exam': 'May 2017', 'question_number': 7}
Observation: {'question_text': 'Explain how compression of data may lead to negative consequences. [3]'}
Thought: I can answer without using any more tools. I'll use the user's language to answer
Answer: Explain how compression of data may lead to negative consequences. [3]
Assistant: Explain how compression of data may lead to negative consequences. [3]

Thought: The current language of the user is: english. I need to use a tool to help me answer the question.
Action: retrieve_question
Action Input: {"exam": "May 2017", "question_number": 7}
Observation: {"question": "Explain how compression of data may lead to negative consequences. [3]", "maximum_points": 3, "markscheme": null}
Thought: I can answer without using any more tools. I'll use the u

KeyboardInterrupt: Interrupted by user